# Cleaning ___npidata___ (Taxonomy Switch Problem)

## 1. Importing Stuff

In [28]:
# importing stuff
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

# showing lots of rows and columns
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

# hiding annoying pink warning boxes
import warnings
warnings.filterwarnings("ignore")

In [29]:
def Get_Taxonomy_Code(df_row):
    j = 1
    switch_code = 0
    while j <= 15:
        column_name = "Healthcare Provider Primary Taxonomy Switch_" + str(j)
        if df_row[column_name] == 'Y':
            switch_code = j
            j = 16
        j+=1
    if switch_code != 0:
        col_1 = "Healthcare Provider Taxonomy Code_" + str(switch_code)
        return df_row[col_1]
    else:
        return -1

In [30]:
db = sqlite3.connect('data/NPPES_Data_Dissemination_February_2021/npidata.sqlite')

In [34]:
from tqdm.notebook import tqdm
for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', chunksize = 10000)):
    chunk = chunk[chunk['Provider Business Practice Location Address State Name'] == 'TN']
    chunk['Provider Business Practice Location Address Postal Code'] = chunk['Provider Business Practice Location Address Postal Code'].astype(int)
    chunk['Healthcare_Provider_Taxonomy_Code'] = chunk.apply(Get_Taxonomy_Code, axis = 1)
    chunk = chunk[chunk['Healthcare_Provider_Taxonomy_Code'] != -1]
    chunk.columns = [x.replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk = chunk[['NPI','Entity_Type_Code','Provider_Organization_Name_(Legal_Business_Name)',
                   'Provider_Last_Name_(Legal_Name)','Provider_First_Name',
                   'Provider_First_Line_Business_Practice_Location_Address',
                   'Provider_Second_Line_Business_Practice_Location_Address',
                   'Provider_Business_Practice_Location_Address_City_Name',
                   'Provider_Business_Practice_Location_Address_Postal_Code',
                   'Provider_Credential_Text','Provider_Gender_Code',
                  'Healthcare_Provider_Taxonomy_Code']]
    chunk = chunk.rename(columns = {"Entity_Type_Code": "Entity_Type_Code",
                                    "Provider_Organization_Name_(Legal_Business_Name)":"Provider_Organization_Name",
                                   "Provider_Last_Name_(Legal_Name)":"Provider_Last_Name",
                                   "Provider_First_Name":"Provider_First_Name",
                                   "Provider_Credential_Text":"Provider_Credential_Text",
                                   "Provider_Gender_Code":"Gender",
                                   'Provider_First_Line_Business_Practice_Location_Address':'First_Line_Address',
                                   'Provider_Second_Line_Business_Practice_Location_Address':'Second_Line_Address',
                                   'Provider_Business_Practice_Location_Address_City_Name':'Address_City',
                                   'Provider_Business_Practice_Location_Address_Postal_Code':'Address_Postal_Codes'})
    chunk.to_sql('npidata', db, if_exists = 'append', index = False)            # Append the chunk to a calls table